In [ ]:
import sys
from PyQt5 import QtCore, QtWidgets, QtWebEngineWidgets
from lxml import html as htmlRenderer

class WebPage(QtWebEngineWidgets.QWebEnginePage):
    def __init__(self):
        super(WebPage, self).__init__()
        self._newsAndComments = []
        self._baseUrl = ""
        self._firstPageProcessed = False;
        self.loadFinished.connect(self.handleLoadFinished)

    def start(self, baseUrl):
#         self._urls = iter(urls)
#         self.fetchNext()
        self._baseUrl = baseUrl
        self.load(QtCore.QUrl(url))

    def fetchNext(self):
        try:
            url = next(self._urls)
        except StopIteration:
            return False
        else:
            self.load(QtCore.QUrl(url))
        return True

    def processCurrentPage(self, html):
        url = self.url().toString()
        renderedPage = htmlRenderer.fromstring(html)
        if (self._firstPageProcessed):
            # do stuff with html...
    #         print('loaded: [%d chars] %s' % (len(html), url))
            print(" -> processing extras URL: {}".format(url))
            autores = renderedPage.xpath("//div[@class='autor']/span[@class='alias']/text()")
            fechas = renderedPage.xpath("//time/span[@class='fecha']/text()")
            horas = renderedPage.xpath("//time/span[@class='hora']/text()")
            comentarios = renderedPage.xpath("//div[@class='texto-comentario']/p/text()")

            if (len(autores) > 0):
                # there are comments
                linksRepe = [url]*(len(autores))
                autores.reverse()
                fechas.reverse()
                horas.reverse()
                comentarios.reverse()

                tuples = zip(linksRepe, autores, fechas, horas, comentarios)
                self._newsAndComments = newsAndComments + tuples
            if not self.fetchNext():
                print(_newsAndComments)
                QtWidgets.qApp.quit()
        else:
            print(" -> processing base URL: {}".format(url))
            links = renderedPage.xpath("//a/@href")
#             links = [ link for link in el if "elmundo" in link and "#ancla_comentarios" not in link and  "logo_home" not in link and "intcmp" not in link and "menu.html" not in link and "cgi.elmundo.es" not in link]
            self._urls = iter(links)
            self._firstPageProcessed = True
            self.fetchNext()

    def handleLoadFinished(self):
        self.toHtml(self.processCurrentPage)


In [ ]:

# generate some test urls
urls = []
url = 'https://www.elmundo.es'
app = QtWidgets.QApplication(sys.argv)
webpage = WebPage()
webpage.start(url)
sys.exit(app.exec_())